# AdaBandFlt Distinguishing Spikes from Bursts or Not

### 01_06_20

## Loading Data

In [2]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
from scipy.signal import *
import matplotlib.pyplot as plt
#from AdaBandFlt import *

%matplotlib tk

In [3]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_new_spike_data/539W6cbaseRaw.txt'


# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')
df

,El 21,El 31,El 41,El 22,El 32,El 42,El 23,El 33,El 43,El 15
%t,,,,,,,,,,
0.00,-2.61,1.02,0.80,1.93,-1.25,-1.59,-0.57,4.09,-1.14,-0.34
0.04,-3.30,0.00,2.05,1.25,-1.02,-0.45,-1.82,2.61,-0.45,-0.80
0.08,-2.73,0.11,2.73,1.48,0.45,1.14,-1.14,3.52,-0.23,-0.80
0.12,-3.07,-0.45,1.25,0.80,-0.57,-0.57,-1.25,2.50,-1.02,-0.57
0.16,-2.39,1.48,0.34,1.14,0.00,-0.57,-0.34,1.48,-0.23,0.45
...,...,...,...,...,...,...,...,...,...,...
20799.80,2.05,-4.43,2.95,0.11,2.39,3.86,-2.27,2.61,2.95,0.34
20799.84,3.64,-5.68,-0.80,-1.70,2.73,0.91,-1.70,0.23,1.59,1.14
20799.88,4.89,-3.41,-3.30,-1.59,0.57,-1.70,-1.82,-0.34,-0.11,1.25


# AdaBandFlt Functions

## Initializing noise levels

In [30]:
def test_valid_window(window, test_level = 5):
    """
    window : the window in the signal that has to be tested
    
    This funtion test the window to insure that it doesn't contain the signal of interest (spike)
    """
    #non zero ?
    second = np.percentile(window, 2)
    thirtyth = np.percentile(window, 30)
    #print(str(second) + "\t" + str(thirtyth) + "\t" + str(second/thirtyth))
    if abs(second/thirtyth) < test_level : 
        return True
    else : 
        return False

In [31]:
def init_noise_levels(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25):
    
    if estimator_type == "RMS":
        return init_noise_levels_RMS(signal, fs, 
                      noise_window_size = noise_window_size,
                      required_valid_windows = required_valid_windows,
                      old_noise_level_propagation = old_noise_level_propagation, 
                      test_level = test_level,
                      percentile_value = percentile_value)
        
    elif estimator_type == "MAD":
        return init_noise_levels_MAD(signal, fs, 
                      noise_window_size = noise_window_size,
                      required_valid_windows = required_valid_windows,
                      old_noise_level_propagation = old_noise_level_propagation, 
                      test_level = test_level,
                      percentile_value = percentile_value)
    
    else: return None
    
    
def init_noise_levels_RMS(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      percentile_value = 25):
    
    nb_valid_windows = 0
    list_RMS = []
    noise_levels = []
    
    noise_level = -1
    
     
    #boucle en indice
#    for window_index in range(0,len(signal)-(len(signal)%int(fs*noise_window_size)),int(fs*noise_window_size)):
    for window_index in range(0,len(signal),int(fs*noise_window_size)):
        test = test_valid_window(signal.iloc[window_index: window_index + int(fs*noise_window_size)], test_level)
        if nb_valid_windows < required_valid_windows:
            if test == True :
                RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                list_RMS.append(RMS)
                nb_valid_windows += 1
            
            if nb_valid_windows == required_valid_windows:
                noise_level = np.percentile(list_RMS, percentile_value)
                for elm in range(0, window_index, int(fs*noise_window_size)):
                    noise_levels.append(noise_level)
                
        else :
            """if test == True:
                if (window + int(fs*noise_window_size)) > (len(signal)-1) :
                    N25 = np.percentile(abs(signal.iloc[window:]), 25)
                else :
                    N25 = np.percentile(abs(signal.iloc[window: window + int(fs*noise_window_size)]), 25)
                noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
            noise_levels.append(noise_level)"""
            if test == True:
                if (window_index + int(fs*noise_window_size)) > (len(signal)-1) :
                    RMS = np.sqrt(np.mean(signal.iloc[window_index:]**2))
                else :
                    RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                list_RMS.append(RMS)
                NX = np.percentile(list_RMS, percentile_value)
                new_noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*NX
                noise_level = new_noise_level
            noise_levels.append(noise_level)
            
    #cas ou il n'y a pas eut 100 fenetres de bruit valides rencontrees
    if noise_level == -1:
        
        #cas ou aucune fenetre valide n'a ete rencontree
        if noise_levels == []:
            for elm in range(0, len(signal), int(fs*noise_window_size)):
                noise_levels.append(0)
            
        else:
            noise_level = np.percentile(list_RMS, percentile_value)
            for elm in range(0, len(signal), int(fs*noise_window_size)):
                noise_levels.append(noise_level)
        
    
    noise_levels.append(noise_level)        
    plt.figure()
    plt.plot(list_RMS)
    plt.xlabel('Time Windows')
    plt.title('RMS values')
    plt.grid(True)
                
    return noise_levels

def init_noise_levels_MAD(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      percentile_value = 25):
    
    nb_valid_windows = 0
    list_MAD = []
    noise_levels = []
     
    #boucle en indice
    for window_index in range(0,len(signal),int(fs*noise_window_size)):
        test = test_valid_window(signal.iloc[window_index: window_index + int(fs*noise_window_size)], test_level)
        if nb_valid_windows < required_valid_windows:
            if test == True :
                ###RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                MAD = scipy.stats.median_absolute_deviation(signal.iloc[window_index: window_index + int(fs*noise_window_size)])
                list_MAD.append(MAD)
                nb_valid_windows += 1
            
            if nb_valid_windows == required_valid_windows:
                noise_level = np.percentile(list_MAD, percentile_value)
                for elm in range(0, window_index, int(fs*noise_window_size)):
                    noise_levels.append(noise_level)
                
        else :
            """if test == True:
                if (window + int(fs*noise_window_size)) > (len(signal)-1) :
                    N25 = np.percentile(abs(signal.iloc[window:]), 25)
                else :
                    N25 = np.percentile(abs(signal.iloc[window: window + int(fs*noise_window_size)]), 25)
                noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
            noise_levels.append(noise_level)"""
            if test == True:
                if (window_index + int(fs*noise_window_size)) > (len(signal)-1) :
                    ###RMS = np.sqrt(np.mean(signal.iloc[window_index:]**2))
                    MAD = scipy.stats.median_absolute_deviation(signal.iloc[window_index:])
                else :
                    ###RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                    MAD = scipy.stats.median_absolute_deviation(signal.iloc[window_index: window_index + int(fs*noise_window_size)])
                list_MAD.append(MAD)
                NX = np.percentile(list_MAD, percentile_value)
                new_noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*NX
                noise_level = new_noise_level
            noise_levels.append(noise_level)
    
    noise_levels.append(noise_levels)        
    plt.figure()
    plt.plot(list_MAD)
    plt.xlabel('Time Windows')
    plt.title('MAD values')
    plt.grid(True)
                
    return noise_levels

## Finding spikes

In [52]:
# find spikes

def find_spike(signal, initial_index, noise_levels, fs, spike_info,
               window_size = 0.002,
               noise_window_size = 0.01,
               threshold_factor = 3.5,
               maxseparation = 0.0008,
               time_checkmaxlocal = 0.0002,
               reduct_factor = 0.6,
               burst_threshold = 10): #facteur de réduction pour le threshold positif
    
    offset_index = int(np.round(signal.index[0]*fs/1000))
    
    if initial_index < len(signal) + offset_index:
        i = initial_index
        for value in signal.iloc[initial_index-offset_index:]:
            threshold = threshold_factor*noise_levels[int(np.round((i/fs)//noise_window_size))]
            if value < -threshold:

                #verifier qu'on est pas encore dans un effet du spike precedant et decaler la fenetre si besoin
                if signal.iloc[initial_index-offset_index-1] < -threshold:
                    for value_ in signal.iloc[initial_index-offset_index:]:
                        if value_ > -threshold:
                            return i
                        i += 1

                indice_1er_depass = i
                while(True):
                    if i < len(signal)+offset_index-1:
                        if signal.iloc[i-offset_index + 1]>signal.iloc[i-offset_index]:
                            break
                        else :
                            i+=1
                    else :
                        break
                             
                #partir à la recherche d'un max du spike a droite
                i_max_right = 'nan'  
                for k in range(int(np.round(maxseparation*fs))):
                    if (i-offset_index + k) < len(signal)-1:
                        if signal.iloc[i-offset_index+k] > reduct_factor*threshold and signal.iloc[i-offset_index+k]>signal.iloc[i-offset_index+k+1]:
                            if checkmaxlocal(signal, "right",i+k,offset_index,int(np.round(time_checkmaxlocal*fs))):
                                i_max_right = i+k
                                break
                #partir à la recherche d'un max du spike a gauche
                i_max_left = 'nan'  
                for k in range(int(np.round(maxseparation*fs))):
                    if (i-offset_index - k) > 0:
                        if signal.iloc[i-offset_index-k] > reduct_factor*threshold and signal.iloc[i-offset_index-k]>signal.iloc[i-offset_index-k-1]:
                            if checkmaxlocal(signal, "left",i-k,offset_index,int(np.round(time_checkmaxlocal*fs))):
                                i_max_left = i-k
                                break
                
                
                if i_max_left == 'nan' and i_max_right == 'nan':
                    #on a pas rencontré de spike
                    while signal.iloc[i-offset_index] < -threshold:
                        i += 1
                    return i
                
                else:
                    #récolte infos du spike
                    amplitude = 0
                    if i_max_left != 'nan' and i_max_right != 'nan':
                        if signal.iloc[i_max_left-offset_index] < signal.iloc[i_max_right-offset_index]:
                            amplitude = signal.iloc[i_max_right-offset_index] - signal.iloc[i-offset_index]
                        else :
                            amplitude = signal.iloc[i_max_left-offset_index] - signal.iloc[i-offset_index]
                    elif i_max_left != 'nan':
                            amplitude = signal.iloc[i_max_left-offset_index] - signal.iloc[i-offset_index]
                    elif i_max_right != 'nan':
                            amplitude = signal.iloc[i_max_right-offset_index] - signal.iloc[i-offset_index]
                    
                    if amplitude >= 2*burst_threshold:
                        is_it_burst = True
                    elif amplitude < 2*burst_threshold:
                        is_it_burst = False
  
                        # indice min, indice 1er depasssement
                        # max gauche, max droite
                        # variation d'amplitude entre min et max
                    
                    spike_info.append([i, indice_1er_depass,
                                        i_max_left, i_max_right,
                                        amplitude,is_it_burst])
                    return indice_1er_depass+int(np.round(window_size*fs))
                
                break  
            i += 1

    return -44

def checkmaxlocal(local_signal, sens, supposed_i_min,offset_index, nb_index_research=3):
    if(sens == "right"):
        k = 0
        while k <= nb_index_research:
        #for k in range(nb_index_research):
            if((local_signal.iloc[supposed_i_min-offset_index + k]) < (local_signal.iloc[supposed_i_min-offset_index + k + 1])):
                return False
            k += 1
        return True
    elif(sens == "left"):
        k = 0
        while k <= nb_index_research:
        #for k in range(nb_index_research):
            if((local_signal.iloc[supposed_i_min-offset_index - k]) < (local_signal.iloc[supposed_i_min-offset_index - k - 1])):
                return False
            k+=1
        return True
    else:
        return False

def find_spikes(signal, noise_levels, fs,
               window_size = 0.002,
               noise_window_size = 0.01,
               threshold_factor = 3.5,
               maxseparation = 0.0008,
               time_checkmaxlocal = 0.0002,
               burst_threshold = 10):
    
    initial = int(np.round(signal.index[0]*fs/1000))
    spike_info = []
    
    while initial != -44:
        initial = find_spike(signal, initial, noise_levels, fs, spike_info,
                             window_size = window_size,
                             noise_window_size = noise_window_size,
                             threshold_factor = threshold_factor,
                             maxseparation = maxseparation,
                             time_checkmaxlocal = time_checkmaxlocal)
        
    df_spike_info = pd.DataFrame(spike_info)
    df_spike_info.columns = ['indice_min', 'indice_1er_depass','indice_max_gauche','indice_max_droite','Delta_amplitudes','burst?']

    return df_spike_info

## Recording the spikes

In [33]:
#record spike

def record_spikes(signal, fs, spike_info,
                  align_method,
                  t_before = 0.002,
                  t_after = 0.002):
    
    if (align_method in spike_info.columns) == False:
        print("align_method is incorrect, please choose one of the following :" + str(spike_info.columns))
        return None
    
    else:
        spike_centers = spike_info[align_method].values
        
    t_b = int(np.round(fs*(t_before)))
    t_a = int(np.round(fs*(t_after)))
    
    data = np.array([[float(x) for x in range(t_b+t_a+1)]])
    
    initial_index = int(np.round(signal.index[0]*fs/1000))
    
    for center in spike_centers:
        if center < t_b + initial_index:
            spike = [0 for i in range(0, t_b-(center-initial_index))]
            spike = np.concatenate((spike, signal.values[:center + t_a - initial_index]))
            data = np.insert(data, len(data), spike, axis=0)
            
        elif center > len(signal)-t_a + initial_index:
            spike = signal.values[center - t_b - initial_index:]
            spike = np.concatenate((spike,[0 for i in range(0, t_a - (len(signal) + initial_index-center))]))
            data = np.insert(data, len(data), spike, axis=0)
            
        else :
            spike = signal.values[center - t_b - initial_index: center + t_a + 1 - initial_index]
            data = np.insert(data, len(data), spike, axis=0)

    print(np.shape(data))
    data = data.transpose()
    spike_data = pd.DataFrame(data)
    
    return spike_data

In [34]:
def record_spikes_oneline(signal, fs, spike_info,
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002):

    if (align_method in spike_info.columns) == False:
        print("align_method is incorrect, please choose one of the following :" + str(spike_info.columns))
        return None
    
    else:
        spike_centers = spike_info[align_method].values
        
    offset_index = int(np.round(signal.index[0]*fs/1000))
    
    t_b = int(np.round(fs*(t_before)))
    t_a = int(np.round(fs*(t_after)))
    
    data = np.array(['NaN' for x in range(len(signal))])
    data = data.astype(float)
    times = np.array(['NaN' for x in range(len(signal))])
    times = times.astype(pd.Timestamp)
    
    for center in spike_centers:
        if center < t_b + offset_index:
            data[:center + t_a - offset_index] = signal.values[:center + t_a - offset_index]
            times[:center + t_a - offset_index] = signal.index[:center + t_a - offset_index]
            
        elif center > len(signal) - t_a + offset_index:
            data[center - t_b - offset_index:] = signal.values[center - t_b - offset_index:]
            times[center - t_b - offset_index:] = signal.index[center - t_b - offset_index:]
            
        else :
            data[center - t_b - offset_index: center + t_a + 1 - offset_index] = signal.values[center - t_b - offset_index: center + t_a + 1 - offset_index]
            times[center - t_b - offset_index: center + t_a + 1 - offset_index] = signal.index[center - t_b - offset_index: center + t_a + 1 - offset_index]

    spike_data_oneline = pd.DataFrame(data, index = times.astype(float))
    
    return spike_data_oneline

## Printing the spikes

In [35]:
from random import randint

def print_spikes(spike_data,
                 t_before_alignement = 0,
                 first_spike = 1,
                 last_spike = -1,
                 fs = 25000,
                 randomize = False,
                 nb_spike = 20,
                 y_lim_min = -50,
                 y_lim_max = 60):
    
    if randomize == True:        
        kept = []
        m = len(spike_data.values[0])
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = spike_data.iloc[:,kept].values
        
    else:
        x = spike_data.iloc[:,first_spike:last_spike]
        
    figure = plt.figure()
    t_b = int(np.round(fs*(t_before_alignement)))
    axes = figure.add_subplot(1, 1, 1)
    axes.plot((spike_data.iloc[:,0]-t_b)*1000/fs, x)
    axes.set_xlabel('Time in ms')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()
    
"""
print_spikes(spike_data,
             t_before_alignement = 0.0015,
             first_spike = 1,
             last_spike = 20,
             fs = 25000,
             y_lim_min = -50,
             y_lim_max = 60)
"""

'\nprint_spikes(spike_data,\n             t_before_alignement = 0.0015,\n             first_spike = 1,\n             last_spike = 20,\n             fs = 25000,\n             y_lim_min = -50,\n             y_lim_max = 60)\n'

In [36]:
def record_spikes_burst_distinction(signal, fs, spike_info,
                  align_method,
                  t_before = 0.002,
                  t_after = 0.002):
    
    spike_data_burst = record_spikes(signal, fs, spike_info.loc[spike_info['burst?'] == True], align_method, t_before = t_before, t_after = t_after)
    spike_data_no_burst = record_spikes(signal, fs, spike_info.loc[spike_info['burst?'] == False], align_method, t_before = t_before, t_after = t_after)
    
    return spike_data_burst, spike_data_no_burst

In [37]:
def record_spikes_oneline_distinction(signal, fs, spike_info,
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002):
    
    spike_data_oneline_burst = record_spikes_oneline(signal, fs, spike_info.loc[spike_info['burst?'] == True], align_method, t_before = t_before, t_after = t_after)
    spike_data_oneline_no_burst = record_spikes_oneline(signal, fs, spike_info.loc[spike_info['burst?'] == False], align_method, t_before = t_before, t_after = t_after)
    
    return spike_data_oneline_burst, spike_data_oneline_no_burst

# Testing the functions

### Global Parameters

In [109]:
size_of_data = 1000000
fs = 50000

method_align = 'indice_1er_depass'
time_before = 0.0015
time_after = 0.0015

threshold_factor = 3.5
maxseparation = 0.0008
reduct_factor = 0.6
burst_threshold = 10 #µV

#n_electrode = 4

### Filtering functions and cutoff frequencies

In [39]:
lowcut = 100.0
highcut = 5000.0

In [40]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [41]:
signal_to_filter = df.iloc[:size_of_data,6] #Electrode 58

y = butter_bandpass_filter(signal_to_filter, lowcut, highcut, fs, order=6)

filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

filtereddf

,0
%t,
0.00,0.258071
0.04,0.486492
0.08,0.542644
0.12,0.316481
0.16,-0.196697
...,...
20799.80,2.688028
20799.84,2.522823
20799.88,2.006812


In [42]:
signal_filtered = filtereddf.iloc[:,0]
signal = filtereddf.iloc[:,0] ##selecting an electrode to use

### Testing...

In [44]:
### PLOTTING SIGNAL:

plt.figure()
plt.plot(df.index, signal, 'g-', linewidth=2, label='data filtered')
plt.xlabel('Time [ms]')
plt.ylabel('Amplitude [µV]')
plt.title('Filtered Signal')
plt.legend()
plt.grid()
plt.show()

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [45]:
noise_levels = init_noise_levels(signal_filtered, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 20,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)

In [46]:
plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5, 1.0, 'Noise Levels')

In [163]:
spike_info = find_spikes(signal, noise_levels, fs,
               window_size = 0.002,
               noise_window_size = 0.01,
               threshold_factor = threshold_factor,
               maxseparation = maxseparation,
               time_checkmaxlocal = 0.0002,
               burst_threshold = burst_threshold)

spike_info.head(30)

,indice_min,indice_1er_depass,indice_max_gauche,indice_max_droite,Delta_amplitudes,burst?
0,2909,2907,nan,2941,10.153078,False
1,17771,17771,17737,nan,9.942149,False
2,20316,20316,20308,nan,10.677421,False
3,53784,53781,nan,53797,13.388152,False
4,54104,54102,nan,54136,12.061430,False
5,54270,54270,nan,54309,14.235471,False
6,54392,54391,54353,nan,16.671837,False
7,54514,54507,nan,54543,18.135680,False
8,54666,54664,nan,54682,16.471992,False
9,55273,55269,55246,nan,22.538088,True


## Spike_distinction_fine_tuning

In [164]:
spike_info.loc[spike_info['burst?'] == True]

16

In [167]:
def spike_fine_tuning(spike_info):
    true_before = spike_info.loc[spike_info['burst?'] == True]
    True_spikes = spike_info.index
    true_before.index = [x for x in range(len(true_before))]
    print(true_before)
    for i in range(len(true_before)-1):
        print("true_before.loc[i+1]['indice_1er_depass']")
        print(true_before.loc[i+1]['indice_1er_depass'])
        print(true_before.loc[i+1]['indice_1er_depass']-true_before.loc[i]['indice_1er_depass'])
        if(true_before.loc[i+1]['indice_1er_depass']-true_before.loc[i]['indice_1er_depass']<1000):
            for j in range(True_spikes[i],True_spikes[i+1]):
                spike_info.at[j, 'burst?'] = True
    
## +5 is ok

In [168]:
spike_fine_tuning(spike_info)
spike_info.loc[spike_info['burst?'] == True]

    indice_min  indice_1er_depass indice_max_gauche indice_max_droite  \
0        17771              17771             17737               nan   
1        20316              20316             20308               nan   
2        53784              53781               nan             53797   
3        54270              54270               nan             54309   
4        54392              54391             54353               nan   
5        55273              55269             55246               nan   
6        56246              56243             56233               nan   
7        56396              56393               nan             56432   
8        58180              58178             58170             58214   
9       205681             205677               nan            205717   
10      206206             206201               nan            206220   
11      206770             206766               nan            206805   
12      207227             207215            207198

,indice_min,indice_1er_depass,indice_max_gauche,indice_max_droite,Delta_amplitudes,burst?
1,17771,17771,17737,nan,9.942149,True
2,20316,20316,20308,nan,10.677421,True
3,53784,53781,nan,53797,13.388152,True
4,54104,54102,nan,54136,12.061430,True
5,54270,54270,nan,54309,14.235471,True
6,54392,54391,54353,nan,16.671837,True
9,55273,55269,55246,nan,22.538088,True
10,56246,56243,56233,nan,17.312860,True
11,56396,56393,nan,56432,14.395789,True
13,58180,58178,58170,58214,14.003686,True


## Recording spikes distinguished

In [153]:
spike_data_burst = record_spikes(signal, fs, spike_info.loc[spike_info['burst?'] == True],
                  method_align,
                  t_before = time_before,
                  t_after = time_after)

(17, 151)


In [154]:
spike_data_no_burst = record_spikes(signal, fs, spike_info.loc[spike_info['burst?'] == False],
                  method_align,
                  t_before = time_before,
                  t_after = time_after)

(96, 151)


In [155]:
spike_data_oneline_burst = record_spikes_oneline(signal, fs, spike_info.loc[spike_info['burst?'] == True],
                  method_align,
                  t_before = time_before,
                  t_after = time_after)

In [156]:
spike_data_oneline_no_burst = record_spikes_oneline(signal, fs, spike_info.loc[spike_info['burst?'] == False],
                  method_align,
                  t_before = time_before,
                  t_after = time_after)

spike_data_oneline = record_spikes_oneline(signal, fs, spike_info,
                  'indice_zero_central',
                  t_before = 0.002,
                  t_after = 0.002)

In [157]:
plt.plot(df.index, signal, color = 'blue')
plt.plot(spike_data_oneline_burst.index, spike_data_oneline_burst, color = 'red')
plt.plot(spike_data_oneline_no_burst.index, spike_data_oneline_no_burst, color = 'green')
plt.title('Filtered Signal with Detected Spikes with RMS')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.legend()
plt.grid(True)

No handles with labels found to put in legend.
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [46]:
print_spikes(spike_data,
             t_before_alignement = time_before,
             first_spike = 1,
             last_spike = 40,
             fs = fs)